# **Dataset Quality**


## **Libraries**

In [1]:
import os
import re
import warnings

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

from utils import constants


## **Directories**

In [2]:
CORPORA_FOLDER = constants.CORPORA_FOLDER
DATASET_DIR = os.path.join(CORPORA_FOLDER, 'jojajovai_all.csv')

In [3]:
dataset = pd.read_csv(DATASET_DIR, sep=',', encoding='utf-8')


## **Corpus**

In [4]:
dataset.head()

,split,source,gn,es,tokens_gn,tokens_es
0,train,abc,"Itaugua omokyre'ÿ ""omopotî"" Congreso","En Itauguá promueven ""limpiar"" el Congreso",Itaugua omokyre'ÿ `` omopotî '' Congreso,En Itauguá promueven `` limpiar '' el Congreso
1,train,abc,"Omopotîvo hikuái tetãme vicio política, ko'ã i...",Con el propósito de limpiar al país de los vic...,"Omopotîvo hikuái tetãme vicio política , ko'ã ...",Con el propósito de limpiar al país de los vic...
2,train,abc,Ko'ã 50 tapicha oñembyaty parroquía Virgen del...,Unas 50 personas se encuentran en la plazoleta...,Ko'ã 50 tapicha oñembyaty parroquía Virgen del...,Unas 50 personas se encuentran en la plazoleta...
3,train,abc,Itaugüeño oipotáva ohechauka ipotîha itáva ha ...,Los itaugüeños quieren demostrar que los parag...,Itaugüeño oipotáva ohechauka ipotîha itáva ha ...,Los itaugüeños quieren demostrar que los parag...
4,train,abc,Opavave tavaygua ojerúre senador Oscar Gonzál...,"Personas de todas las edades, pero en su mayor...",Opavave tavaygua ojerúre senador Oscar Gonzále...,"Personas de todas las edades , pero en su mayo..."


In [5]:
SPLIT_COLUMN = 'split'
SOURCE_COLUMN = 'source'
LANGUAGE_SOURCE_COLUMN = 'gn'
LANGUAGE_TARGET_COLUMN = 'es'

In [6]:
train_dataset = dataset[dataset[SPLIT_COLUMN] == 'train']

In [7]:
len(train_dataset)

20207

## **Quality metrics**

- The minimum and maximum length of segments
- The ratio between the source and target lengths.
- The ratio between alphabet to non-alphabet characters
- The ratio between alphabet to non-alphabet words

Source: https://marian-nmt.github.io/examples/training-overview

In [8]:
def sentence_word_length(sentence: str):
    return len(sentence.split())

def sentence_character_length(sentence: str):
    return len(sentence)

def sentences_word_length_ratio(sentence1: str, sentence2: str):
    return sentence_word_length(sentence1) / sentence_word_length(sentence2)

def sentences_character_length_ratio(sentence1: str, sentence2: str):
    return sentence_character_length(sentence1) / sentence_character_length(sentence2)

def sentence_alphabet_character_ratio(sentence: str):
    non_alphabet_characters = re.sub('[a-zA-Z]+', '', sentence)
    non_alphabet_characters = re.sub(' ( )+', '', non_alphabet_characters)
    non_alphabet_characters = non_alphabet_characters.strip()
    return len(non_alphabet_characters) / len(sentence)

def sentence_alphabet_words_ratio(sentence: str):
    words = sentence.split()
    non_alphabet_words = list(filter(lambda word: not word.isalpha(), words))
    non_alphabet_words = ' '.join(non_alphabet_words)
    non_alphabet_words = re.sub(' ( )+', '', non_alphabet_words)
    non_alphabet_words = non_alphabet_words.strip()
    return len(non_alphabet_words.split()) / len(sentence.split())

In [9]:
def create_quality_metrics_df(df: pd.DataFrame) -> pd.DataFrame:
    warnings.filterwarnings('ignore')
    df['source_sentence_word_length'] = df[LANGUAGE_SOURCE_COLUMN].apply(sentence_word_length)
    df['source_sentence_character_length'] = df[LANGUAGE_SOURCE_COLUMN].apply(sentence_character_length)
    df['source_sentence_alphabet_character_ratio'] = df[LANGUAGE_SOURCE_COLUMN].apply(sentence_alphabet_character_ratio)
    df['source_sentence_alphabet_words_ratio'] = df[LANGUAGE_SOURCE_COLUMN].apply(sentence_alphabet_words_ratio)
    df['target_sentence_word_length'] = df[LANGUAGE_TARGET_COLUMN].apply(sentence_word_length)
    df['target_sentence_character_length'] = df[LANGUAGE_TARGET_COLUMN].apply(sentence_character_length)
    df['target_sentence_alphabet_character_ratio'] = df[LANGUAGE_TARGET_COLUMN].apply(sentence_alphabet_character_ratio)
    df['target_sentence_alphabet_words_ratio'] = df[LANGUAGE_TARGET_COLUMN].apply(sentence_alphabet_words_ratio)
    df['sentence_word_length_ratio'] = df.apply(lambda row: sentences_word_length_ratio(row[LANGUAGE_SOURCE_COLUMN], row[LANGUAGE_TARGET_COLUMN]), axis=1)
    df['sentence_character_length_ratio'] = df.apply(lambda row: sentences_character_length_ratio(row[LANGUAGE_SOURCE_COLUMN], row[LANGUAGE_TARGET_COLUMN]), axis=1)
    warnings.filterwarnings('default')
    return df

In [10]:
quality_metrics_df = create_quality_metrics_df(train_dataset)

In [27]:
quality_metrics_df['source_sentence_word_length'].sum(), quality_metrics_df['target_sentence_word_length'].sum()

(309920, 459629)

In [12]:
quality_metrics_df.tail()

,split,source,gn,es,tokens_gn,tokens_es,source_sentence_word_length,source_sentence_character_length,source_sentence_alphabet_character_ratio,source_sentence_alphabet_words_ratio,target_sentence_word_length,target_sentence_character_length,target_sentence_alphabet_character_ratio,target_sentence_alphabet_words_ratio,sentence_word_length_ratio,sentence_character_length_ratio
20202,train,spl,Ko cursillo reieténte ojejapokuaa ha oikóta 18...,El cursillo es totalmente gratuito y se desarr...,Ko cursillo reieténte ojejapokuaa ha oikóta 18...,El cursillo es totalmente gratuito y se desarr...,20,137,0.145985,0.300000,28,160,0.087500,0.142857,0.714286,0.856250
20203,train,spl,"Upe rire, oñepyrũta tekombo’e ojeikévo jasyrun...",Las clases iniciarán en la primera semana de a...,"Upe rire , oñepyrũta tekombo'e ojeikévo jasyru...",Las clases iniciarán en la primera semana de a...,18,131,0.251908,0.388889,20,110,0.127273,0.200000,0.900000,1.190909
20204,train,spl,Reñemomaranduvekuaa ko’ápe (0981) 463-338.,Para mayor información llamar al (0981) 463-338.,Reñemomaranduvekuaa ko'ápe ( 0981 ) 463-338 .,Para mayor información llamar al ( 0981 ) 463-...,4,42,0.476190,0.750000,7,48,0.333333,0.285714,0.571429,0.875000
20205,train,spl,Ko guarani ñe’ẽme jehekombo’e rupive hi’ã temi...,La carrera en Lengua Guaraní tiene por objetiv...,Ko guarani ñe'ẽme jehekombo'e rupive hi'ã temi...,La carrera en Lengua Guaraní tiene por objetiv...,24,190,0.184211,0.500000,33,196,0.035714,0.121212,0.727273,0.969388
20206,train,spl,Umi oikeséva oñemoaradu oiporukuaava’erã vaiva...,Los postulantes deberán demostrar un manejo bá...,Umi oikeséva oñemoaradu oiporukuaava'erã vaiva...,Los postulantes deberán demostrar un manejo bá...,7,61,0.114754,0.285714,9,63,0.063492,0.111111,0.777778,0.968254


In [13]:
quality_metrics_df.describe()

,source_sentence_word_length,source_sentence_character_length,source_sentence_alphabet_character_ratio,source_sentence_alphabet_words_ratio,target_sentence_word_length,target_sentence_character_length,target_sentence_alphabet_character_ratio,target_sentence_alphabet_words_ratio,sentence_word_length_ratio,sentence_character_length_ratio
count,20207.000000,20207.000000,20207.000000,20207.000000,20207.000000,20207.000000,20207.000000,20207.000000,20207.000000,20207.000000
mean,15.337259,118.303657,0.146058,0.279846,22.746029,141.127431,0.058637,0.126050,0.724802,0.892976
std,10.926488,84.373576,0.070045,0.176456,16.214508,101.808352,0.045188,0.111748,0.224360,0.256727
min,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.122449,0.159509
25%,7.000000,51.000000,0.100335,0.176471,9.000000,56.000000,0.032258,0.062500,0.600000,0.762556
50%,14.000000,104.000000,0.138889,0.266667,21.000000,126.000000,0.050388,0.111111,0.690476,0.857143
75%,22.000000,169.000000,0.183673,0.368421,33.000000,205.000000,0.075000,0.166667,0.814815,0.977142
max,155.000000,1172.000000,1.000000,1.000000,214.000000,1303.000000,0.666667,1.000000,8.000000,8.750000


### **Sentence length comparison**

In [14]:
sentence_word_length_ratio_df = quality_metrics_df[quality_metrics_df['sentence_word_length_ratio'] >= 2][[LANGUAGE_SOURCE_COLUMN, LANGUAGE_TARGET_COLUMN, 'sentence_word_length_ratio']].sort_values(by='sentence_word_length_ratio', ascending=False)
sentence_word_length_ratio_df

,gn,es,sentence_word_length_ratio
19709,"Estigarribia Tavao kotýpe, ypykuéra Lumnanas-p...",Estigarribia.,8.0
16708,pysyrõ (oñepysyrõ se defiende),defender,4.0
16673,ñemitỹ ha mymbakuéra rehegua,agropecuario,4.0
14515,Ñeꞌẽveve og̃uahẽva ojejerureꞌỹ rehe,Spam,4.0
20084,"Upéicha, opa umi 100 ñe’ẽme oñembohasámava Moz...",299 nuevas traducciones en el último mes repor...,3.0
...,...,...,...
14317,Yvytu pepóre,Online,2.0
14277,Firefox Aguaratata,Mozilla,2.0
14276,Pumbyry popegua,Móviles,2.0
14223,Paraguái ñeꞌẽ Paraguái ñeꞌẽ ñeꞌẽveve rysýi,Lista de correo,2.0


In [15]:
for index, row in sentence_word_length_ratio_df.iterrows():
    print('#' * 36 + f'| Index: {index} ' + '|' + f' Ratio: {row["sentence_word_length_ratio"]} |' + '#' * 35)
    print(f'{row[LANGUAGE_SOURCE_COLUMN]}')
    print(f'{row[LANGUAGE_TARGET_COLUMN]}')
    print('#' * 100)
    print('\n')

####################################| Index: 19709 | Ratio: 8.0 |###################################
Estigarribia Tavao kotýpe, ypykuéra Lumnanas-pevoi, comunidad Abizai peguakuérape.
Estigarribia.
####################################################################################################


####################################| Index: 16708 | Ratio: 4.0 |###################################
pysyrõ (oñepysyrõ se defiende)
defender
####################################################################################################


####################################| Index: 16673 | Ratio: 4.0 |###################################
ñemitỹ ha mymbakuéra rehegua
agropecuario
####################################################################################################


####################################| Index: 14515 | Ratio: 4.0 |###################################
Ñeꞌẽveve og̃uahẽva ojejerureꞌỹ rehe
Spam
###################################################################

In [16]:
sentence_character_length_ratio_df = quality_metrics_df[quality_metrics_df['sentence_character_length_ratio'] >= 2][[LANGUAGE_SOURCE_COLUMN, LANGUAGE_TARGET_COLUMN, 'sentence_character_length_ratio']].sort_values(by='sentence_character_length_ratio', ascending=False)
sentence_character_length_ratio_df

,gn,es,sentence_character_length_ratio
14515,Ñeꞌẽveve og̃uahẽva ojejerureꞌỹ rehe,Spam,8.750000
13946,Karameg̃ua apytuꞌũ guasu,CPU,8.000000
19709,"Estigarribia Tavao kotýpe, ypykuéra Lumnanas-p...",Estigarribia.,6.307692
14676,tekorenda ¿mba'épa neremiandu?,Estado,5.000000
14970,"maitei, mba'éichapa",Hola,4.750000
...,...,...,...
12022,Taupéicha.,Amén.,2.000000
11752,KO ÁRA,HOY,2.000000
15194,hetave,Más,2.000000
15261,ha,Y,2.000000


In [17]:
for index, row in sentence_character_length_ratio_df.iterrows():
    print('#' * 36 + f'| Index: {index} ' + '|' + f' Ratio: {round(row["sentence_character_length_ratio"], 2)} |' + '#' * 35)
    print(f'{row[LANGUAGE_SOURCE_COLUMN]}')
    print(f'{row[LANGUAGE_TARGET_COLUMN]}')
    print('#' * 100)
    print('\n')

####################################| Index: 14515 | Ratio: 8.75 |###################################
Ñeꞌẽveve og̃uahẽva ojejerureꞌỹ rehe
Spam
####################################################################################################


####################################| Index: 13946 | Ratio: 8.0 |###################################
Karameg̃ua apytuꞌũ guasu
CPU
####################################################################################################


####################################| Index: 19709 | Ratio: 6.31 |###################################
Estigarribia Tavao kotýpe, ypykuéra Lumnanas-pevoi, comunidad Abizai peguakuérape.
Estigarribia.
####################################################################################################


####################################| Index: 14676 | Ratio: 5.0 |###################################
tekorenda ¿mba'épa neremiandu?
Estado
################################################################################

### **Non-alphabetic characters rate**

In [18]:
quality_metrics_df[quality_metrics_df['source_sentence_alphabet_character_ratio'] > 0.5][[LANGUAGE_SOURCE_COLUMN, LANGUAGE_TARGET_COLUMN, 'source_sentence_alphabet_character_ratio']].sort_values(by='source_sentence_alphabet_character_ratio', ascending=False)

,gn,es,source_sentence_alphabet_character_ratio
11961,Ã,Alma,1.000000
13133,– Hẽẽ!.,– ¡Síii!.,0.857143
12705,"20 légua, 200 térâ 1.000,","veinte leguas, doscientas o mil,",0.760000
14144,Ñeꞌẽ,Idioma,0.750000
14204,Ñeꞌẽ,Lengua,0.750000
14532,Ñeꞌẽrã,Temas,0.666667
14689,Ñe'ẽrã,Título,0.666667
16804,ñe’ẽme’ẽ,promesa,0.625000
685,Péva adhesión G. 10.000.000.,De acuerdo a los datos la adhesión es G. 10.00...,0.607143
16700,ñoñe’ẽme’ẽ,contrato,0.600000


In [19]:
quality_metrics_df[quality_metrics_df['target_sentence_alphabet_character_ratio'] >= 0.5][[LANGUAGE_SOURCE_COLUMN, LANGUAGE_TARGET_COLUMN, 'target_sentence_alphabet_character_ratio']].sort_values(by='target_sentence_alphabet_character_ratio', ascending=False)

,gn,es,target_sentence_alphabet_character_ratio
13133,– Hẽẽ!.,– ¡Síii!.,0.666667
13584,heẽ,sí.,0.666667
14611,Mbaꞌépa,¿Qué?,0.600000
13590,Ára 1 (19 Jasypo),Día 1 (Viernes 19) .,0.550000
20111,Oñemba’eporanduse terã ojeike hag̃ua (0983) 33...,Consultas e inscripción en (0983) 336 322 / (0...,0.504762
12416,“IKATÚRÔ”,“SI”,0.500000
13552,ohota,iré.,0.500000


### **Absolute individual length**

**Words**

In [20]:
source_sentence_word_length = quality_metrics_df[quality_metrics_df['source_sentence_word_length'] > 50][[LANGUAGE_SOURCE_COLUMN, LANGUAGE_TARGET_COLUMN, 'source_sentence_word_length']].sort_values(by='source_sentence_word_length', ascending=False)
source_sentence_word_length

,gn,es,source_sentence_word_length
15384,Umi oñe’ẽva’ekue apytépe ojejuhu: Mgtr . Ladis...,"Entre los expositores, resaltamos la presencia...",155
16908,Opa umi ñe’ẽyvoty oĩva ko arandukápe:Ayvu Rapy...,Los textos poéticos reunidos en la antología s...,126
17423,Paraguái Ñe’ẽnguéra Sãmbyhyha (PÑS) oporohenói...,La Secretaría de Políticas Lingüísticas (SPL) ...,124
15382,Ambue tetãgui ou oñe’ẽvo ko’ã tapicha arandu: ...,Como ponentes internacionales han participado ...,112
15627,Umi 300 tapicha iko’iva’ekue Paraguái tuichaku...,Los casi 300 participantes procedentes de prác...,95
...,...,...,...
11547,"Ko ára, Ministerio de la Niñez y la Adolescenc...","Este martes, el Ministerio de la Niñez y la Ad...",51
12444,Ka’avove’i rakâmimi ojegueru ha ojekutujereva’...,"Esas ramas son clavadas en el suelo, formando ...",51
17402,"Ág̃a arakõi 3 jasypoteĩme, 17 aravo jave, Koty...","Para el día lunes 3 de junio, a partir de las ...",51
15991,Ateneo de Lengua y Cultura Guarani oreko hína ...,El Ateneo de Lengua y Cultura Guaraní posee un...,51


In [21]:
for index, row in source_sentence_word_length.iterrows():
    print('#' * 36 + f'| Index: {index} ' + '|' + f' Length: {row["source_sentence_word_length"]} |' + '#' * 35)
    print(f'{row[LANGUAGE_SOURCE_COLUMN]}')
    print(f'{row[LANGUAGE_TARGET_COLUMN]}')
    print('#' * 100)
    print('\n')

####################################| Index: 15384 | Length: 155 |###################################
Umi oñe’ẽva’ekue apytépe ojejuhu: Mgtr . Ladislaa Alcaraz de Silvero, Paraguái Ñe’ẽnguéra Sãmbyhyha Sãmbyhyhára; Lic . Susy Delgado Ñe’ẽpapára, haihára, Premio Nacional de Literatura 2017-pe; Tkj . Carlos Ferreira Quiñónez, Guarani Ñe’ẽ Rerekuapavẽ Yta; Dra . María Eva Mansfeld, Comisión Nacional de Bilingüismo-ygua; Tkj . David Galeano Olivera, Ateneo de Lengua y Cultura Guaraní Moakãharapavẽ; Lic . César González Martínez, Dirección General de Educación Escolar Indígena MEC-pegua Moakãhára; Mgtr . Graciela Aquino, Instituto Superior de Lenguas Universidad Nacional de Asunción pegua Guarani Mbo’esyry coordinadora; Tkj . Dionisio Fleitas, Universidad Nacional Itapúa pegua mbo’ehára ha tembikuaarekahára CONACYT-pe; Lic . Mario Rubén Álvarez, Momaranduhára, haihára ha ñe’ẽmboasahára; Lic . Mirta Martínez, moranduhára ha Tetã Rekuái ñanduti renda mbojerahára guaraníme; Mgtr . Federico Rol

In [22]:
target_sentence_word_length = quality_metrics_df[quality_metrics_df['target_sentence_word_length'] > 75][[LANGUAGE_SOURCE_COLUMN, LANGUAGE_TARGET_COLUMN, 'target_sentence_word_length']].sort_values(by='target_sentence_word_length', ascending=False)
target_sentence_word_length

,gn,es,target_sentence_word_length
15384,Umi oñe’ẽva’ekue apytépe ojejuhu: Mgtr . Ladis...,"Entre los expositores, resaltamos la presencia...",214
15627,Umi 300 tapicha iko’iva’ekue Paraguái tuichaku...,Los casi 300 participantes procedentes de prác...,153
16908,Opa umi ñe’ẽyvoty oĩva ko arandukápe:Ayvu Rapy...,Los textos poéticos reunidos en la antología s...,135
15382,Ambue tetãgui ou oñe’ẽvo ko’ã tapicha arandu: ...,Como ponentes internacionales han participado ...,130
17423,Paraguái Ñe’ẽnguéra Sãmbyhyha (PÑS) oporohenói...,La Secretaría de Políticas Lingüísticas (SPL) ...,129
...,...,...,...
3125,"Péicha, opyta sin efecto resolución Nº 132 ára...","De esta manera, se deja sin efecto la resoluci...",76
5994,Hacienda omyesakã periodo de transición gubern...,Hacienda explica que dado el periodo de transi...,76
7779,Péva dirección general Derechos Humanos Minis...,A través de la dirección general de Derechos H...,76
8151,"Ramón Herebia, director casa de estudio-gua, h...","Ramón Herebia, director de la casa de estudio,...",76


In [23]:
for index, row in target_sentence_word_length.iterrows():
    print('#' * 36 + f'| Index: {index} ' + '|' + f' Length: {row["target_sentence_word_length"]} |' + '#' * 35)
    print(f'{row[LANGUAGE_SOURCE_COLUMN]}')
    print(f'{row[LANGUAGE_TARGET_COLUMN]}')
    print('#' * 100)
    print('\n')

####################################| Index: 15384 | Length: 214 |###################################
Umi oñe’ẽva’ekue apytépe ojejuhu: Mgtr . Ladislaa Alcaraz de Silvero, Paraguái Ñe’ẽnguéra Sãmbyhyha Sãmbyhyhára; Lic . Susy Delgado Ñe’ẽpapára, haihára, Premio Nacional de Literatura 2017-pe; Tkj . Carlos Ferreira Quiñónez, Guarani Ñe’ẽ Rerekuapavẽ Yta; Dra . María Eva Mansfeld, Comisión Nacional de Bilingüismo-ygua; Tkj . David Galeano Olivera, Ateneo de Lengua y Cultura Guaraní Moakãharapavẽ; Lic . César González Martínez, Dirección General de Educación Escolar Indígena MEC-pegua Moakãhára; Mgtr . Graciela Aquino, Instituto Superior de Lenguas Universidad Nacional de Asunción pegua Guarani Mbo’esyry coordinadora; Tkj . Dionisio Fleitas, Universidad Nacional Itapúa pegua mbo’ehára ha tembikuaarekahára CONACYT-pe; Lic . Mario Rubén Álvarez, Momaranduhára, haihára ha ñe’ẽmboasahára; Lic . Mirta Martínez, moranduhára ha Tetã Rekuái ñanduti renda mbojerahára guaraníme; Mgtr . Federico Rol

**Characters**

In [24]:
quality_metrics_df[quality_metrics_df['source_sentence_character_length'] > 500][[LANGUAGE_SOURCE_COLUMN, LANGUAGE_TARGET_COLUMN, 'source_sentence_character_length']].sort_values(by='source_sentence_character_length', ascending=False)

,gn,es,source_sentence_character_length
15384,Umi oñe’ẽva’ekue apytépe ojejuhu: Mgtr . Ladis...,"Entre los expositores, resaltamos la presencia...",1172
16908,Opa umi ñe’ẽyvoty oĩva ko arandukápe:Ayvu Rapy...,Los textos poéticos reunidos en la antología s...,1008
17423,Paraguái Ñe’ẽnguéra Sãmbyhyha (PÑS) oporohenói...,La Secretaría de Políticas Lingüísticas (SPL) ...,890
15382,Ambue tetãgui ou oñe’ẽvo ko’ã tapicha arandu: ...,Como ponentes internacionales han participado ...,785
15627,Umi 300 tapicha iko’iva’ekue Paraguái tuichaku...,Los casi 300 participantes procedentes de prác...,759
16015,Jepéramo oĩ gueteri heta henda ijetu’uha Faceb...,Si bien existen aún ciertas dificultades para ...,684
8412,Valenzuela oikéva ko’ã grupo oîháme salesiano ...,Valenzuela formará parte de este grupo con el ...,679
15628,Upe Amandajépe avei ojejerure toñemoheñói Guar...,Dicho Congreso también recomendó la creación d...,653
7495,"Pohanohára Juan Ramón Bueno Jara, abogado fren...","El doctor Juan Ramón Bueno Jara, abogado de lo...",583
2991,"Abogado, Cristóbal Cáceres, ogueraháva ko káso...","Su abogado, Cristóbal Cáceres, dijo a los medi...",559


In [25]:
quality_metrics_df[quality_metrics_df['target_sentence_character_length'] > 750][[LANGUAGE_SOURCE_COLUMN, LANGUAGE_TARGET_COLUMN, 'target_sentence_character_length']].sort_values(by='target_sentence_character_length', ascending=False)

,gn,es,target_sentence_character_length
15384,Umi oñe’ẽva’ekue apytépe ojejuhu: Mgtr . Ladis...,"Entre los expositores, resaltamos la presencia...",1303
15627,Umi 300 tapicha iko’iva’ekue Paraguái tuichaku...,Los casi 300 participantes procedentes de prác...,1025
17423,Paraguái Ñe’ẽnguéra Sãmbyhyha (PÑS) oporohenói...,La Secretaría de Políticas Lingüísticas (SPL) ...,879
16908,Opa umi ñe’ẽyvoty oĩva ko arandukápe:Ayvu Rapy...,Los textos poéticos reunidos en la antología s...,866
15382,Ambue tetãgui ou oñe’ẽvo ko’ã tapicha arandu: ...,Como ponentes internacionales han participado ...,848
8064,Oñemotenondévo manifestación tavaygua ombosako...,Siguiendo la serie de manifestaciones de ciuda...,757
15628,Upe Amandajépe avei ojejerure toñemoheñói Guar...,Dicho Congreso también recomendó la creación d...,752
